In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import random
import math
import time
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
import operator 
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import datetime
from datetime import date
import operator
from matplotlib import pyplot
import rpy2 as r
from pandas import DataFrame
from pandas import concat
from statsmodels.tsa.api import ExponentialSmoothing

In [2]:
confirmed_cases_global = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
coloco = ['Lat','Long']
confirmed_cases_global.drop(coloco, inplace=True, axis=1)
confirmed_cases_global=confirmed_cases_global[confirmed_cases_global['Province/State']!='Recovered']
confirmed_cases_global=confirmed_cases_global[confirmed_cases_global['Province/State']!='Diamond Princess']
confirmed_cases_global.head()

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20
0,NaN,Afghanistan,0,0,0,0,0,0,0,0,...,16509,17267,18054,18969,19551,20342,20917,21459,22142,22890
1,NaN,Albania,0,0,0,0,0,0,0,0,...,1164,1184,1197,1212,1232,1246,1263,1299,1341,1385
2,NaN,Algeria,0,0,0,0,0,0,0,0,...,9626,9733,9831,9935,10050,10154,10265,10382,10484,10589
3,NaN,Andorra,0,0,0,0,0,0,0,0,...,844,851,852,852,852,852,852,852,852,852
4,NaN,Angola,0,0,0,0,0,0,0,0,...,86,86,86,86,88,91,92,96,113,118


In [3]:
confirmed_cases_Canada=confirmed_cases_global[confirmed_cases_global['Country/Region']=='Canada']
confirmed_cases_Canada=confirmed_cases_Canada.groupby(['Country/Region']).sum().reset_index()
confirmed_cases_Canada.insert(0, 'Province/State', float('NaN'))
confirmed_cases_Canada

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20
0,NaN,Canada,0,0,0,0,1,1,2,2,...,93960,94641,95269,95947,96475,97178,97779,98241,98720,99159


In [4]:
confirmed_cases_US = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
confirmed_cases_US.rename(columns={'Province_State':'Province/State', 'Country_Region':'Country/Region', 'Long_':'Long' }, inplace=True)
coloc = ['UID', 'iso2','iso3' ,'code3', 'FIPS','Admin2','Combined_Key','Lat','Long']
confirmed_cases_US.drop(coloc, inplace=True, axis=1)
confirmed_cases_US.head()

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20
0,American Samoa,US,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Guam,US,0,0,0,0,0,0,0,0,...,175,177,179,179,179,179,179,180,180,183
2,Northern Mariana Islands,US,0,0,0,0,0,0,0,0,...,23,24,26,26,26,27,28,30,30,30
3,Puerto Rico,US,0,0,0,0,0,0,0,0,...,3935,4023,4508,4620,4915,4985,5046,5185,5329,5352
4,Virgin Islands,US,0,0,0,0,0,0,0,0,...,70,70,71,71,71,71,71,71,72,72


In [5]:
confirmed_cases_US=confirmed_cases_US.groupby(['Province/State','Country/Region']).sum().reset_index()

In [6]:
deaths_cases_global = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
colo = ['Lat','Long']
deaths_cases_global=deaths_cases_global[deaths_cases_global['Province/State']!='Recovered']
deaths_cases_global=deaths_cases_global[deaths_cases_global['Province/State']!='Diamond Princess']
deaths_cases_global.drop(colo, inplace=True, axis=1)
deaths_cases_global.head()

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20
0,NaN,Afghanistan,0,0,0,0,0,0,0,0,...,270,294,300,309,327,357,369,384,405,426
1,NaN,Albania,0,0,0,0,0,0,0,0,...,33,33,33,33,34,34,34,34,34,35
2,NaN,Algeria,0,0,0,0,0,0,0,0,...,667,673,681,690,698,707,715,724,732,741
3,NaN,Andorra,0,0,0,0,0,0,0,0,...,51,51,51,51,51,51,51,51,51,51
4,NaN,Angola,0,0,0,0,0,0,0,0,...,4,4,4,4,4,4,4,4,4,5


In [7]:
deaths_cases_Canada=deaths_cases_global[deaths_cases_global['Country/Region']=='Canada']
deaths_cases_Canada=deaths_cases_Canada.groupby(['Country/Region']).sum().reset_index()
deaths_cases_Canada.insert(0, 'Province/State', float('NaN'))
deaths_cases_Canada

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20
0,NaN,Canada,0,0,0,0,0,0,0,0,...,7477,7578,7716,7777,7849,7876,7909,7969,8037,8070


In [8]:
deaths_cases_US = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
deaths_cases_US.rename(columns={'Province_State':'Province/State', 'Country_Region':'Country/Region','Long_':'Long' }, inplace=True)
col = ['UID', 'iso2','iso3' ,'code3', 'FIPS','Admin2','Combined_Key', 'Population','Long','Lat']
deaths_cases_US.drop(col, inplace=True, axis=1)
deaths_cases_US.head()

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20
0,American Samoa,US,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Guam,US,0,0,0,0,0,0,0,0,...,5,5,5,5,5,5,5,5,5,5
2,Northern Mariana Islands,US,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2
3,Puerto Rico,US,0,0,0,0,0,0,0,0,...,138,140,140,141,142,142,142,142,143,144
4,Virgin Islands,US,0,0,0,0,0,0,0,0,...,6,6,6,6,6,6,6,6,6,6


In [9]:
deaths_cases_US = deaths_cases_US.groupby(['Province/State','Country/Region']).sum().reset_index()

In [10]:
confirmed_cases= pd.concat([confirmed_cases_global, confirmed_cases_US,confirmed_cases_Canada], axis=0, sort=False)

In [11]:
deaths_cases = pd.concat([deaths_cases_global, deaths_cases_US,deaths_cases_Canada], axis=0, sort=False)

In [12]:
recoveries_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
coloco = ['Lat','Long']
recoveries_cases.drop(coloco, inplace=True, axis=1)
recoveries_cases.head()

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20
0,NaN,Afghanistan,0,0,0,0,0,0,0,0,...,1450,1522,1585,1762,1830,1875,2171,2651,3013,3326
1,NaN,Albania,0,0,0,0,0,0,0,0,...,891,898,898,910,925,938,945,960,980,1001
2,NaN,Algeria,0,0,0,0,0,0,0,0,...,6067,6218,6297,6453,6631,6717,6799,6951,7074,7255
3,NaN,Andorra,0,0,0,0,0,0,0,0,...,733,735,738,741,741,744,751,757,759,780
4,NaN,Angola,0,0,0,0,0,0,0,0,...,18,18,18,21,24,24,38,38,40,41


In [13]:
colnames = confirmed_cases.columns.values

In [14]:
colonames = recoveries_cases.columns.values

In [15]:
confirmed = confirmed_cases.loc[:, colnames[4]:colnames[-1]]
deaths = deaths_cases.loc[:, colnames[4]:colnames[-1]]
recoveries = recoveries_cases.loc[:, colonames[4]:colonames[-1]]

In [16]:
time_date = confirmed.columns.values

In [17]:
all_days = np.array([i for i in range(len(time_date))]).reshape(-1, 1)

## Make Predictions with Double Exponential Smoothing


In [18]:
pred_days_ahead = 30
predictions_ahead = np.array([i for i in range(len(time_date)+pred_days_ahead)]).reshape(-1, 1)

predictions_ahead_plus_day2 = np.array([i for i in range(len(time_date)+2+1)]).reshape(-1, 1)
predictions_ahead_plus_day7 = np.array([i for i in range(len(time_date)+7+1)]).reshape(-1, 1)
predictions_ahead_plus_day30 = np.array([i for i in range(len(time_date)+30+1)]).reshape(-1, 1)

In [19]:
start = datetime.datetime.strptime(colnames[4], "%m/%d/%y")
end = datetime.datetime.strptime(colnames[-1], "%m/%d/%y")
predictions_ahead_dates2 = [start + datetime.timedelta(days=d) for d in range(len(predictions_ahead_plus_day2))]
predictions_ahead_dates7 = [start + datetime.timedelta(days=d) for d in range(len(predictions_ahead_plus_day7))]
predictions_ahead_dates30 = [start + datetime.timedelta(days=d) for d in range(len(predictions_ahead_plus_day30))]

In [20]:
confirmed_cases = confirmed_cases[~confirmed_cases['Country/Region'].isin(['Tajikistan','Comoros'])]

In [21]:
countries=confirmed_cases['Country/Region'].tolist()
province=confirmed_cases['Province/State'].tolist()

In [22]:
'Tajikistan' in countries

False

In [23]:
'Comoros' in countries

False

In [25]:
def pred_with_exp_smoothing(timeseries):

    #timeseries.index.freq = 'D'
    timeseries.index = pd.DatetimeIndex(timeseries.index).to_period('D')
    timeseries.freq = 'D'
    #idx=add_freq(timeseries.index, freq='D')
    model=ExponentialSmoothing(timeseries,trend='add',damped=True)
    
    md=model.fit()
    
    forecasts=md.forecast(30) #forecast for 30 days, keep 2,7,30 simultaneously
    print("forecasts")
    #print(forecasts)
    return forecasts   # Series!! of  all forecasts for 30 days


In [26]:
def add_freq(idx, freq=None):
    """Add a frequency attribute to idx, through inference or directly.

    Returns a copy.  If `freq` is None, it is inferred.
    """

    idx = idx.copy()
    if freq is None:
        if idx.freq is None:
            freq = pd.infer_freq(idx)
        else:
            return idx
    idx.freq = pd.tseries.frequencies.to_offset(freq)
    if idx.freq is None:
        raise AttributeError('no discernible frequency found to `idx`.  Specify'
                             ' a frequency string with `freq`.')
    return idx

In [27]:
def Exp_Smooth_Analysis(region):
   #create a dataframe for timeseries analysis
    date_rng = pd.date_range(start=colnames[4], end=colnames[-1], freq='D') #creates the days
    df = pd.DataFrame(date_rng, columns=['date'])
    df['cases']=np.transpose(region) #this is a dataframe with dates and cases.
    #df['date']=add_freq(df['date'], freq='D')
    df = df.set_index('date')
    df=df[df['cases'] != 0] #remove zeroes, they do not help for analysis. 
    if (len(df)>1):
        if(df['cases'][-1]-df['cases'][0]>5):
            #print("test1")
            pred_ts=pred_with_exp_smoothing(df['cases']) #this is an array converted to a list
        else:
            #print('test2')
            pred_ts=[df['cases'][-1]+1]*30
        #pred_ts.index=df.index  
    else:
        pred_ts=np.zeros(30)
    return pred_ts


In [30]:
import operator
def subpopulation_Analysis(cases,country,province, all_days,subpopulation, pred_days_ahead=30):
    
    pred  = list()
    if pd.isnull(province):
        region = cases.loc[(cases['Country/Region'] == country) & (pd.isnull(cases['Province/State']))]
    else :
        region = cases.loc[(cases['Country/Region'] == country) & (cases['Province/State'] == province)]
    print(region)
    
    reg =  np.array(region.loc[:, colnames[4]:colnames[-1]])
    region =  np.array(region.loc[:, colnames[4]:colnames[-1]]).reshape(-1, 1)
        
    #train_x, test_x, train_y, test_y = model_selection.train_test_split(all_days,region,test_size = 0.04,random_state = 42,shuffle=False)
    
    #pred_svm =  SVM_regression(train_x, train_y.ravel(), test_x, test_y, predictions_ahead,subpopulation,all_days,region.ravel())
    #pred.append(pred_svm)
    
    #TODO: a for loop
    daily_increase_threshold1=np.absolute((reg[-1][-1]-reg[-1][-2])/reg[-1][-2])*100
    daily_increase_threshold2=np.absolute((reg[-1][-2]-reg[-1][-3])/reg[-1][-3])*100
    daily_increase_threshold3=np.absolute((reg[-1][-3]-reg[-1][-4])/reg[-1][-4])*100
    daily_increase_threshold4=np.absolute((reg[-1][-4]-reg[-1][-5])/reg[-1][-5])*100
    daily_increase_bound=1000
    daily_increase_threshold = max(daily_increase_threshold1,daily_increase_threshold2,daily_increase_threshold3,daily_increase_threshold4)
    
    if (daily_increase_threshold>daily_increase_bound):
        best_pred_ahead2 = 0 
        best_pred_ahead7 = 0 
        best_pred_ahead30 = 0 
        region = region
        best_pred = 0
        #best_conf_int = np.zeros(2)
        reject_country = 1
        return(best_pred_ahead2,best_pred_ahead7,best_pred_ahead30, region, best_pred, reject_country)
     
    
    pred_arima=Exp_Smooth_Analysis(reg)
    #pred.append(pred_svm)
    #pred_RF = RF_regression(train_x, train_y.ravel(), test_x, test_y, predictions_ahead,subpopulation,all_days,region.ravel())
    #pred.append(pred_RF)

    
   # pred_gb =  Grad_Boost_regression(train_x, train_y.ravel(), test_x, test_y, predictions_ahead,subpopulation,all_days,region.ravel())
   # pred.append(pred_gb)


    if (subpopulation == 'confirmed'): 
        if (len(pred_arima)==0):
            min_idx, (min_key, min_val) = min(enumerate(prediction_models_confirmed.items()), key=lambda x: x[1][1])
            best_pred = pred[min_idx]
            #conf_int=np.zeros(2)
        else:
            best_pred=pred_arima
            #conf_int=conf_int
    elif (subpopulation == 'deaths'):
        if (len(pred_arima)==0):
            min_idx, (min_key, min_val) = min(enumerate(prediction_models_deaths.items()), key=lambda x: x[1][1])
            best_pred = pred[min_idx]
            #conf_int=np.zeros(2)
        else:
            best_pred=pred_arima
            #conf_int=conf_int
    else :
        if (len(pred_arima)==0):
            min_idx, (min_key, min_val) = min(enumerate(prediction_models_recoveries.items()), key=lambda x: x[1][1])
            best_pred = pred[min_idx]
            #conf_int=np.zeros(2)
        else:
            best_pred=pred_arima
            #conf_int=conf_int
        
    #print(best_pred)
    #print(type(best_pred))
    best_pred_ahead2 = best_pred[1]
    #print("gntm2")
    #print(best_pred_ahead2)
    best_pred_ahead7= best_pred[6]
    #print("gntm7")
    #print(best_pred_ahead7)
    best_pred_ahead30= best_pred[-1]
    #print("gntm30")
    #print(best_pred_ahead30)
    #best_conf_int=conf_int[-1]
   
    #best_conf_int[0]=max(reg[-1][-1], best_conf_int[0])
    #if (best_conf_int[0]>best_conf_int[1]):
        #best_conf_int[1]=2*reg[-1][-1]

    if(best_pred_ahead2<reg[-1][-1]):
        best_pred_ahead2= reg[-1][-1]+2
    
    if(best_pred_ahead7<reg[-1][-1]):
        best_pred_ahead7= reg[-1][-1]+7   
    
    if(best_pred_ahead7<reg[-1][-1]):
        best_pred_ahead7= reg[-1][-1]+30     
    
    reject_country = 0
    
    return best_pred_ahead2,best_pred_ahead7,best_pred_ahead30, region, best_pred, reject_country

In [31]:
pred_file2 = str(2)+"day_prediction_exp_smooth" +  date.today().strftime('%Y-%m-%d') + ".csv"
pred_file2

'2day_prediction_exp_smooth2020-06-12.csv'

In [32]:
pred_file7 = str(7)+"day_prediction_exp_smooth" + date.today().strftime('%Y-%m-%d') + ".csv"
pred_file7

'7day_prediction_exp_smooth2020-06-12.csv'

In [33]:
pred_file30 = str(30)+"day_prediction_exp_smooth" + date.today().strftime('%Y-%m-%d') + ".csv"
pred_file30

'30day_prediction_exp_smooth2020-06-12.csv'

In [34]:

columns = ['Province/State','Country','Target/Date','N','low95N','high95N',
          'R','low95R','high95R','D','low95D','high95D','T','low95T','high95T',
           'M','low95M','high95M','C','low95C','high95C']

            

k = 0
for co in range(len(countries)):
    
    ignore_recoveries = 0    
    
    total_predictions2 = pd.DataFrame(columns = columns)
    total_predictions7 = pd.DataFrame(columns = columns)
    total_predictions30 = pd.DataFrame(columns = columns)
    
    k = k + 1
   
       
    best_pred_confirmed2,best_pred_confirmed7,best_pred_confirmed30, region_con, best_pred_con, reject_country = subpopulation_Analysis(confirmed_cases,countries[co],province[co],
                                                 all_days,"confirmed",pred_days_ahead)
    
    if (reject_country==1):
        print('rejected_conf')
        continue
        
    best_pred_deaths2,best_pred_deaths7,best_pred_deaths30, region_d, best_pred_d, reject_country = subpopulation_Analysis(deaths_cases,countries[co],province[co],
                                              all_days,"deaths",pred_days_ahead)
    if (reject_country==1):
        print('rejected_d')
        continue
        
    if ((countries[co] == 'US' or countries[co] == 'Canada' ) & (pd.notnull(province[co]))):
        ignore_recoveries=1   
    else:
        best_pred_recoveries2,best_pred_recoveries7,best_pred_recoveries30, region_r, best_pred_r, reject_country = subpopulation_Analysis(recoveries_cases,countries[co],province[co],
                                                  all_days,"recoveries",pred_days_ahead)
    
    if (reject_country==1):
        print('rejected_r')
        continue
    
    # Mortality rate
    #if (conf_int_con[0]==0):
       # mort_pred=0
        #mort_pred_low=0
        #mort_pred_up=0
    #else:
    mort_pred2  = best_pred_deaths2/best_pred_confirmed2
    mort_pred7  = best_pred_deaths7/best_pred_confirmed7
    mort_pred30  = best_pred_deaths30/best_pred_confirmed30
        #mort_pred_low= conf_int_d[0]/conf_int_con[1]
        #mort_pred_up=conf_int_d[1]/conf_int_con[0]
    
    #fix province
    province_name=''
    if (pd.notnull(province[co])) :
        province_name=province[co]
    
    if (ignore_recoveries==0):
        R_value2=int(best_pred_recoveries2) 
        R_value7=int(best_pred_recoveries7) 
        R_value30=int(best_pred_recoveries30) 
        #low_R=int(conf_int_r[0])
        #high_R=int(conf_int_r[1])
    else:
        R_value2=''
        R_value7=''
        R_value30=''
        #low_R=''
        #high_R=''
        
    #Critical Cases:
    #Assume that 5% of all cases are critical and need hospitalization:
    serious_percentage=0.05
    critical2=serious_percentage*best_pred_confirmed2
    critical7=serious_percentage*best_pred_confirmed7
    critical30=serious_percentage*best_pred_confirmed30
    #low_critical=serious_percentage*conf_int_con[0]
    #high_critical=serious_percentage*conf_int_con[1]
    
    total_predictions2 = total_predictions2.append({'Province/State': province_name,
                                                'Country': countries[co],
                                                'Target/Date': predictions_ahead_dates2[-1].strftime('%Y-%m-%d'), 
                                                'N': int(best_pred_confirmed2) ,
                                                'low95N':'',
                                                'high95N': '',
                                                'R': R_value2 ,
                                                'low95R' : '',
                                                'high95R' : '',
                                                'D': int(best_pred_deaths2) ,
                                                'low95D' : '',
                                                'high95D': '',
                                                'T': '',
                                                'low95T' : '' ,
                                                'high95T' : '',
                                                'M': mort_pred2,
                                                'low95M': '',
                                                'high95M': '' ,
                                                'C': int(critical2) ,
                                                'low95C':  '' ,
                                                'high95C': '' },ignore_index = True)
        
    total_predictions7 = total_predictions7.append({'Province/State': province_name,
                                                'Country': countries[co],
                                                'Target/Date': predictions_ahead_dates7[-1].strftime('%Y-%m-%d'), 
                                                'N': int(best_pred_confirmed7) ,
                                                'low95N':'',
                                                'high95N': '',
                                                'R': R_value7 ,
                                                'low95R' : '',
                                                'high95R' : '',
                                                'D': int(best_pred_deaths7) ,
                                                'low95D' : '',
                                                'high95D': '',
                                                'T': '',
                                                'low95T' : '' ,
                                                'high95T' : '',
                                                'M': mort_pred7,
                                                'low95M': '',
                                                'high95M': '' ,
                                                'C': int(critical7) ,
                                                'low95C':  '' ,
                                                'high95C': '' },ignore_index = True)
            
    total_predictions30 = total_predictions30.append({'Province/State': province_name,
                                                'Country': countries[co],
                                                'Target/Date': predictions_ahead_dates30[-1].strftime('%Y-%m-%d'), 
                                                'N': int(best_pred_confirmed30) ,
                                                'low95N':'',
                                                'high95N': '',
                                                'R': R_value30 ,
                                                'low95R' : '',
                                                'high95R' : '',
                                                'D': int(best_pred_deaths30) ,
                                                'low95D' : '',
                                                'high95D': '',
                                                'T': '',
                                                'low95T' : '' ,
                                                'high95T' : '',
                                                'M': mort_pred30,
                                                'low95M': '',
                                                'high95M': '' ,
                                                'C': int(critical30) ,
                                                'low95C':  '' ,
                                                'high95C': '' },ignore_index = True)
    
    if (k==1):
        header = True
    else:
        header = False
    
    total_predictions2.to_csv(pred_file2, sep=',',index = False,  mode='a', header = header)
    total_predictions7.to_csv(pred_file7, sep=',',index = False,  mode='a', header = header)
    total_predictions30.to_csv(pred_file30, sep=',',index = False,  mode='a', header = header)
    

  Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
0            NaN    Afghanistan        0        0        0        0        0   

   1/27/20  1/28/20  1/29/20  ...  6/2/20  6/3/20  6/4/20  6/5/20  6/6/20  \
0        0        0        0  ...   16509   17267   18054   18969   19551   

   6/7/20  6/8/20  6/9/20  6/10/20  6/11/20  
0   20342   20917   21459    22142    22890  

[1 rows x 144 columns]
forecasts


C:\Users\user\AppData\Roaming\Python\Python37\site-packages\statsmodels\tsa\holtwinters.py:731: RuntimeWarning: invalid value encountered in greater_equal
  loc = initial_p >= ub


  Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
0            NaN    Afghanistan        0        0        0        0        0   

   1/27/20  1/28/20  1/29/20  ...  6/2/20  6/3/20  6/4/20  6/5/20  6/6/20  \
0        0        0        0  ...     270     294     300     309     327   

   6/7/20  6/8/20  6/9/20  6/10/20  6/11/20  
0     357     369     384      405      426  

[1 rows x 144 columns]
forecasts
  Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
0            NaN    Afghanistan        0        0        0        0        0   

   1/27/20  1/28/20  1/29/20  ...  6/2/20  6/3/20  6/4/20  6/5/20  6/6/20  \
0        0        0        0  ...    1450    1522    1585    1762    1830   

   6/7/20  6/8/20  6/9/20  6/10/20  6/11/20  
0    1875    2171    2651     3013     3326  

[1 rows x 144 columns]
forecasts
  Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
1            NaN        Albania     

C:\Users\user\AppData\Roaming\Python\Python37\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)


forecasts
  Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
2            NaN        Algeria        0        0        0        0        0   

   1/27/20  1/28/20  1/29/20  ...  6/2/20  6/3/20  6/4/20  6/5/20  6/6/20  \
2        0        0        0  ...    6067    6218    6297    6453    6631   

   6/7/20  6/8/20  6/9/20  6/10/20  6/11/20  
2    6717    6799    6951     7074     7255  

[1 rows x 144 columns]
forecasts
  Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
3            NaN        Andorra        0        0        0        0        0   

   1/27/20  1/28/20  1/29/20  ...  6/2/20  6/3/20  6/4/20  6/5/20  6/6/20  \
3        0        0        0  ...     844     851     852     852     852   

   6/7/20  6/8/20  6/9/20  6/10/20  6/11/20  
3     852     852     852      852      852  

[1 rows x 144 columns]
forecasts
  Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
3            NaN        An

...

C:\Users\user\AppData\Roaming\Python\Python37\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)


   Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
36       New York             US        0        0        0        0        0   

    1/27/20  1/28/20  1/29/20  ...  6/2/20  6/3/20  6/4/20  6/5/20  6/6/20  \
36        0        0        0  ...   29968   30019   30174   30236   30280   

    6/7/20  6/8/20  6/9/20  6/10/20  6/11/20  
36   30374   30417   30458    30542    30580  

[1 rows x 144 columns]
forecasts
    Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  \
37  North Carolina             US        0        0        0        0   

    1/26/20  1/27/20  1/28/20  1/29/20  ...  6/2/20  6/3/20  6/4/20  6/5/20  \
37        0        0        0        0  ...   30023   31276   32075   33295   

    6/6/20  6/7/20  6/8/20  6/9/20  6/10/20  6/11/20  
37   34718   35625   36517   37227    38473    39584  

[1 rows x 144 columns]
forecasts
    Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  \
37  North Carolina           

C:\Users\user\AppData\Roaming\Python\Python37\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)


forecasts
   Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
40           Ohio             US        0        0        0        0        0   

    1/27/20  1/28/20  1/29/20  ...  6/2/20  6/3/20  6/4/20  6/5/20  6/6/20  \
40        0        0        0  ...    2259    2300    2340    2357    2372   

    6/7/20  6/8/20  6/9/20  6/10/20  6/11/20  
40    2379    2406    2423     2459     2492  

[1 rows x 144 columns]
forecasts
   Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
41       Oklahoma             US        0        0        0        0        0   

    1/27/20  1/28/20  1/29/20  ...  6/2/20  6/3/20  6/4/20  6/5/20  6/6/20  \
41        0        0        0  ...    6692    6805    6907    7007    7059   

    6/7/20  6/8/20  6/9/20  6/10/20  6/11/20  
41    7150    7205    7363     7483     7626  

[1 rows x 144 columns]
forecasts
   Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
41       Okla

   Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
50           Utah             US        0        0        0        0        0   

    1/27/20  1/28/20  1/29/20  ...  6/2/20  6/3/20  6/4/20  6/5/20  6/6/20  \
50        0        0        0  ...   10202   10497   10813   11252   11798   

    6/7/20  6/8/20  6/9/20  6/10/20  6/11/20  
50   12066   12322   12559    12864    13252  

[1 rows x 144 columns]
forecasts
   Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
50           Utah             US        0        0        0        0        0   

    1/27/20  1/28/20  1/29/20  ...  6/2/20  6/3/20  6/4/20  6/5/20  6/6/20  \
50        0        0        0  ...     113     117     117     120     121   

    6/7/20  6/8/20  6/9/20  6/10/20  6/11/20  
50     121     124     127      128      131  

[1 rows x 144 columns]
forecasts
   Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
51        Vermont      